In [ ]:
!pip install google-api-python-client oauth2client

In [ ]:
import os
import re
from google.colab import auth
from oauth2client.client import GoogleCredentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/documents']

markdown_text = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
"""

In [ ]:
class MarkdownToGoogleDocs:
    def __init__(self):
        self.service = None
        self.footer_style = {
            'backgroundColor': {'color': {'rgbColor': {'red': 0.95, 'green': 0.95, 'blue': 0.95}}},
            'foregroundColor': {'color': {'rgbColor': {'red': 0.4, 'green': 0.4, 'blue': 0.4}}}
        }

    def authenticate(self):
        try:
            auth.authenticate_user()
            creds = GoogleCredentials.get_application_default()
            self.service = build('docs', 'v1', credentials=creds)
            return True
        except Exception as e:
            print(f"Authentication error: {str(e)}")
            return False

    def create_document(self, title):
        try:
            document = self.service.documents().create(body={'title': title}).execute()
            return document.get('documentId')
        except HttpError as e:
            print(f"Error creating document: {str(e)}")
            return None

    def parse_markdown(self, markdown_text):
        requests = []
        current_index = 1
        lines = markdown_text.split('\n')
        in_footer = False
        in_metadata = False

        for i, line in enumerate(lines):
            if line.startswith('#'):
                level = len(re.match(r'^#+', line).group())
                text = line.lstrip('#').strip()
                
                requests.append({
                    'insertText': {
                        'location': {'index': current_index},
                        'text': text + '\n'
                    }
                })
                
                style_type = 'HEADING_1' if level == 1 else 'HEADING_2' if level == 2 else 'HEADING_3'
                requests.append({
                    'updateParagraphStyle': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(text)
                        },
                        'paragraphStyle': {
                            'namedStyleType': style_type,
                            'spaceAbove': {'magnitude': 12, 'unit': 'PT'},
                            'spaceBelow': {'magnitude': 12, 'unit': 'PT'}
                        },
                        'fields': 'namedStyleType,spaceAbove,spaceBelow'
                    }
                })
                
                current_index += len(text) + 1
                continue

            checkbox_match = re.match(r'^(\s*)-\s*\[([ x])\]\s*(.+)$', line)
            if checkbox_match:
                indent_level = len(checkbox_match.group(1)) // 2
                checked = checkbox_match.group(2) == 'x'
                text = checkbox_match.group(3)
                
                checkbox_symbol = '☒' if checked else '☐'
                requests.append({
                    'insertText': {
                        'location': {'index': current_index},
                        'text': f"{checkbox_symbol} {text}\n"
                    }
                })

                requests.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + 1
                        },
                        'textStyle': {
                            'foregroundColor': {
                                'color': {'rgbColor': {'red': 0.2, 'green': 0.2, 'blue': 0.2}}
                            }
                        },
                        'fields': 'foregroundColor'
                    }
                })

                mentions = re.finditer(r'@(\w+)', text)
                for mention in mentions:
                    mention_start = current_index + 2 + mention.start()
                    mention_end = current_index + 2 + mention.end()
                    requests.append({
                        'updateTextStyle': {
                            'range': {
                                'startIndex': mention_start,
                                'endIndex': mention_end
                            },
                            'textStyle': {
                                'foregroundColor': {
                                    'color': {'rgbColor': {'red': 0.2, 'green': 0.4, 'blue': 0.8}}
                                },
                                'bold': True
                            },
                            'fields': 'foregroundColor,bold'
                        }
                    })

                if indent_level > 0:
                    requests.append({
                        'updateParagraphStyle': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(checkbox_symbol) + len(text) + 1
                            },
                            'paragraphStyle': {
                                'indentStart': {'magnitude': 36 * indent_level, 'unit': 'PT'}
                            },
                            'fields': 'indentStart'
                        }
                    })

                current_index += len(checkbox_symbol) + len(text) + 2
                continue

            if line.strip().startswith(('*', '-')):
                indent_match = re.match(r'^(\s*)[*-]\s+(.+)$', line)
                if indent_match:
                    indent_level = len(indent_match.group(1)) // 2
                    text = indent_match.group(2)

                    requests.append({
                        'insertText': {
                            'location': {'index': current_index},
                            'text': text + '\n'
                        }
                    })

                    requests.append({
                        'createParagraphBullets': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(text)
                            },
                            'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
                        }
                    })

                    if indent_level > 0:
                        requests.append({
                            'updateParagraphStyle': {
                                'range': {
                                    'startIndex': current_index,
                                    'endIndex': current_index + len(text)
                                },
                                'paragraphStyle': {
                                    'indentStart': {'magnitude': 36 * indent_level, 'unit': 'PT'}
                                },
                                'fields': 'indentStart'
                            }
                        })

                    current_index += len(text) + 1
                    continue

            if line.strip() == '---':
                requests.append({
                    'insertText': {
                        'location': {'index': current_index},
                        'text': '\n\n'
                    }
                })
                current_index += 2
                in_metadata = True
                continue

            if in_metadata and line.strip():
                requests.append({
                    'insertText': {
                        'location': {'index': current_index},
                        'text': line.strip() + '\n'
                    }
                })
                
                requests.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(line.strip())
                        },
                        'textStyle': {
                            'foregroundColor': {'color': {'rgbColor': {'red': 0.4, 'green': 0.4, 'blue': 0.4}}},
                            'fontSize': {'magnitude': 10, 'unit': 'PT'},
                            'italic': True
                        },
                        'fields': 'foregroundColor,fontSize,italic'
                    }
                })
                
                current_index += len(line.strip()) + 1
                continue

            if line.strip().startswith('## Notes'):
                in_footer = True

            if in_footer and line.strip():
                requests.append({
                    'insertText': {
                        'location': {'index': current_index},
                        'text': line.strip() + '\n'
                    }
                })

                if not line.startswith('#'):
                    requests.append({
                        'updateTextStyle': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(line.strip())
                            },
                            'textStyle': self.footer_style,
                            'fields': 'backgroundColor,foregroundColor'
                        }
                    })

                current_index += len(line.strip()) + 1
                continue

            if line.strip():
                requests.append({
                    'insertText': {
                        'location': {'index': current_index},
                        'text': line.strip() + '\n'
                    }
                })
                current_index += len(line.strip()) + 1

        return requests

    def update_document(self, document_id, markdown_text):
        try:
            requests = self.parse_markdown(markdown_text)
            if requests:
                self.service.documents().batchUpdate(
                    documentId=document_id,
                    body={'requests': requests}
                ).execute()
            return True
        except HttpError as e:
            print(f"Error updating document: {str(e)}")
            return False

In [ ]:
def main():
    converter = MarkdownToGoogleDocs()

    if converter.authenticate():
        doc_id = converter.create_document("Product Team Sync")
        if doc_id:
            if converter.update_document(doc_id, markdown_text):
                print(f"Successfully created document: https://docs.google.com/document/d/{doc_id}")
            else:
                print("Failed to update document")
        else:
            print("Failed to create document")
    else:
        print("Authentication failed")

if __name__ == '__main__':
    main()